In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
# Define the reference function
def custom_function(x):
    return np.sin(x) + np.cos(x)

In [3]:
class ArrayNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ArrayNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
# Example usage:
input_size = 5
hidden_size = 10
output_size = 5

# Create instance of the network
net = ArrayNet(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [5]:

# Generate some dummy data using the user-defined function
numOfdata = 100
input_data = torch.randn(numOfdata, input_size, device=net.device)  # 100 samples, 5 features each
target_data = torch.zeros(numOfdata, output_size, device=net.device)
for i in range(numOfdata):
    target_data[i] = torch.from_numpy(custom_function(input_data[i].cpu().numpy())).to(net.device)

In [6]:
# Train the network
num_epochs = 5000
for epoch in range(num_epochs):
    # Forward pass
    outputs = net(input_data)
    loss = criterion(outputs, target_data)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/5000], Loss: 0.6902
Epoch [200/5000], Loss: 0.5638
Epoch [300/5000], Loss: 0.5090
Epoch [400/5000], Loss: 0.4726
Epoch [500/5000], Loss: 0.4427
Epoch [600/5000], Loss: 0.4157
Epoch [700/5000], Loss: 0.3901
Epoch [800/5000], Loss: 0.3656
Epoch [900/5000], Loss: 0.3413
Epoch [1000/5000], Loss: 0.3175
Epoch [1100/5000], Loss: 0.2960
Epoch [1200/5000], Loss: 0.2773
Epoch [1300/5000], Loss: 0.2617
Epoch [1400/5000], Loss: 0.2489
Epoch [1500/5000], Loss: 0.2380
Epoch [1600/5000], Loss: 0.2290
Epoch [1700/5000], Loss: 0.2215
Epoch [1800/5000], Loss: 0.2150
Epoch [1900/5000], Loss: 0.2092
Epoch [2000/5000], Loss: 0.2038
Epoch [2100/5000], Loss: 0.1988
Epoch [2200/5000], Loss: 0.1941
Epoch [2300/5000], Loss: 0.1896
Epoch [2400/5000], Loss: 0.1854
Epoch [2500/5000], Loss: 0.1816
Epoch [2600/5000], Loss: 0.1780
Epoch [2700/5000], Loss: 0.1745
Epoch [2800/5000], Loss: 0.1713
Epoch [2900/5000], Loss: 0.1682
Epoch [3000/5000], Loss: 0.1654
Epoch [3100/5000], Loss: 0.1626
Epoch [3200/5000]

In [7]:
# Assuming 'net' is the trained network from the previous code snippet

# Generate new input data for prediction
new_input_data = torch.randn(1, input_size, device=net.device)  # 1 sample, 5 features each

# generate test
test_res = custom_function(new_input_data[0].cpu().numpy())

print(test_res)

# Put the network in evaluation mode
net.eval()

[-0.13275892  0.07326698  0.8941484   1.4132845   1.4051492 ]


ArrayNet(
  (fc1): Linear(in_features=5, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=5, bias=True)
)

In [8]:
# Make prediction
with torch.no_grad():
    prediction = net(new_input_data)

print("Prediction:", prediction)

Prediction: tensor([[-0.1756,  0.2064,  0.8282,  0.9926,  1.4063]], device='cuda:0')


In [9]:
# tracing
traced_script_module = torch.jit.trace(net, new_input_data)

In [10]:
# Assuming 'net' is the trained network from the previous code snippet

# Specify the file path where you want to save the model
if net.device.type == 'cuda':
    model_path = "array_net_model_cuda.pt"
    torch.jit.save(traced_script_module, model_path)

    # convert the cuda model to cpu, and save it again.
    traced_script_module_cpu = traced_script_module.to('cpu')
    model_path_cpu = "array_net_model_cpu.pt"
    torch.jit.save(traced_script_module_cpu, model_path_cpu)
    print(f"Model saved to {model_path} and {model_path_cpu}")
else:
    model_path = "array_net_model_cpu.pt"
    torch.jit.save(traced_script_module, model_path)    
    print(f"Model saved to {model_path}")

Model saved to array_net_model_cuda.pt and array_net_model_cpu.pt
